In [3]:
#Loading some libraries
!pip install catboost
import tensorflow as tf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, precision_recall_curve
from catboost import Pool, CatBoostClassifier

from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations
from statsmodels.stats.proportion import proportion_confint

#for visualization
import plotly.express as px

%matplotlib inline
# figure size in inches
rcParams['figure.figsize'] = 14,6

# Loading data into the kernel

In [7]:
import os
os.getcwd()
Path(r"C:\Users\Prudhvi\Downloads\Github projects\data-science-projects\Lending Club").cd()

NameError: name 'Path' is not defined

In [4]:
#Loading the data
file = "C:/Users/Prudhvi/loan18-19.csv"
df1=pd.read_csv(file,parse_dates=['issue_d'], infer_datetime_format=True)   #dataset of accepted loans from 2018-2019



FileNotFoundError: [Errno 2] File C:/Users/Prudhvi/loan18-19.csv does not exist: 'C:/Users/Prudhvi/loan18-19.csv'

In [ ]:
df1.info()

The size of the dataframe is too large for analysis so it's necessary to reduce the amount of datapoints.

In [ ]:

df1.head()
df1.reset_index(drop=True)        #resetting the index


In [ ]:
df1.describe()

## Getting dictionary of names 

In [ ]:
inv_view = pd.read_excel("C:/Users/Prudhvi/LCD.xlsx",
                             sheet_name=1)
inv_view.head(20)
inv_view.info()
inv_feat = inv_view['BrowseNotesFile'].dropna().values
inv_feat = [re.sub('(?<![0-9_])(?=[A-Z0-9])', '_', x).lower().strip() for x in inv_feat]  #the column value names are different from the original dataset since this is a view shown to investors only whereas original dataset has more columns 


## Lending kart data has columns of data which are relevant only after loan is issued so removing those which aren't available at the time of investing

Lending Kart helps connect borrowers with investors.There are certain variables in data which are available once the loan has been disbursed.

We are going to take only variables which are useful before loan has been given to borrowers.This will help us to predict factors which are responsible for good loans.The way it generally works is that borrowers with high credit score get lower interest rates.

But investors will get less returns with lower interest rates so they are interested in loans with higher interest rates, which are classified as risky due to which they are having higher interest rates.So it would be a useful proposition for the business to classify loans accurately aomng high interest loans.

To get a list of such variables we can see their dictionary sheet in which they have given investor columns view.Here we are trying to predict good loans amoung high risk/high interest rate loans. 

In [ ]:
#checking for different columns in both sets, the view to investor and view of the company
feat=df1.columns.values
np.setdiff1d(inv_feat,feat)


In [ ]:
np.setdiff1d(feat,inv_feat)   #checking for values in array 1 which aren't in array 2

# we see that some columns are there in both but spellings are different so changing the spellings

In [ ]:
wrong = ['is_inc_v', 'mths_since_most_recent_inq', 'mths_since_oldest_il_open',
         'mths_since_recent_loan_delinq', 'verified_status_joint']
correct = ['verification_status', 'mths_since_recent_inq', 'mo_sin_old_il_acct',
           'mths_since_recent_bc_dlq', 'verification_status_joint']

In [ ]:
inv_feat = np.setdiff1d(inv_feat, wrong)   #subtracting wrong columns from the dataset 
inv_feat = np.append(inv_feat, correct)    #appending correct values below
inv_feat1 = np.intersect1d(inv_feat, feat)
df2= df1[inv_feat1].copy()
df2.info()  #intersection of columns values available for the investors
df2.select_dtypes('object').head()  #checking if there are any datatypes assigned wrongly as strings

In [ ]:
df2.select_dtypes('object').head()  #checking if there are any datatypes assigned wrongly as strings

1)Need to change 'earliest_cr_line' & 'sec_app_earliest_cr_line' from strings to dates

2)emp_length and id are numeric and their type should be changed to float. In case of emp_length I replace the extreme cases of "< 1 year" and "10+ years" with "0 years" and "11 years" respectively to separate these groups from the rest.

In [ ]:
df2['earliest_cr_line'] = pd.to_datetime(df2['earliest_cr_line'], infer_datetime_format=True)
df2['sec_app_earliest_cr_line'] = pd.to_datetime(df2['sec_app_earliest_cr_line'], infer_datetime_format=True)

df2['emp_length'] = df2['emp_length'].replace({'< 1 year': '0 years', '10+ years': '11 years'})
df2['emp_length'] = df2['emp_length'].str.extract('(\d+)').astype('float')
df2['id'] = df2['id'].astype('float')


## EDA of loan amount

In [ ]:
df1['int_round'] = df1['int_rate'].round(0).astype(int)  #rounding the interest rates
df1.info()
#creating a figure to plot on

loan_amount = df1["loan_amnt"].values
funded_amount = df1["funded_amnt"].values
investor_funds = df1["funded_amnt_inv"].values

plt.figure(figsize=(20,14))

#plotting distribution of loan amount
plt.subplot(211)
g = sns.distplot(loan_amount)
g.set_xlabel("Loan Amount Value", fontsize=16)
g.set_ylabel("Frequency", fontsize=16)
g.set_title("Loan Amount Distribuition", fontsize=20)

#plotting the overall fulfillment rate by the company of the loans requested
plt.figure(figsize=(20,14))

plt.subplot(212)
f = sns.distplot(funded_amount)
f.set_xlabel("Funded amount value", fontsize=16)
f.set_ylabel("Frequency", fontsize=16)
f.set_title("Funded Amount Distribution", fontsize=20)

#plotting the distribution of loan amount fulfillment by investors
plt.figure(figsize=(20,14))

plt.subplot(212)
h = sns.distplot(investor_funds)
h.set_xlabel("Investor funded amount value", fontsize=16)
h.set_ylabel("Frequency", fontsize=16)
h.set_title("Investor Funded Amount Distribution", fontsize=20)

df1['int_round'].describe()
total = len(df1)
plt.figure(figsize=(20,16))
plt.subplot(212)
g1 = sns.countplot(x="int_round", data=df1, 
                  color='blue')
g1.set_xlabel("Loan Interest Rate", fontsize=16)
g1.set_ylabel("Count", fontsize=16)
g1.set_title("Interest Rate Distribuition", fontsize=20)
sizes=[] # Get highest values in y
for p in g1.patches:
    height = p.get_height()
    sizes.append(height)
    g1.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=12) 
g1.set_ylim(0, max(sizes) * 1.10) # set y limit based on highest heights

plt.subplots_adjust(hspace = 0.4,top = 0.9)

plt.show()

## Checking for missing values

Once we are done with a basic analysis of the numerical variables like loan,interest etc we are going to check if there are any missing values in the data.

In [ ]:
nan_mean = df2.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean

In [ ]:
df2 = df2.drop(['desc', 'member_id'], axis=1, errors='ignore')

Removing these two columns as they are completely empty so let's remove them as there are only missing values in them

## Handling Missing values
Taking care of missing values in the above columns by filling them appropriately.
There are some categorical variables which can be filled with empty strings.To check which ones are categorical variables we can check in the cleaned dataset and take a difference of sets twice.

In [ ]:
f=list(nan_mean.index)
f
cat_cols=df2.select_dtypes('object').columns.values
cat_cols

In [ ]:
#finding out which are categorical variables from f
set1=np.setdiff1d(f,cat_cols)  #getting list of numerical variables from f by subtracting the categorical columns from cleaned df2
set2=np.setdiff1d(f,set1)     #subtracting the list of numerical from f to get back the categorical columns left in this f
set2                         #using these to set the empty string to these categorical columns missing values

We find that there are two categorical variables,so their missing values can be filled by an empty string

In [ ]:
fill_empty = ['emp_title', 'verification_status_joint']
fill_max = ['bc_open_to_buy', 'mo_sin_old_il_acct', 'mths_since_last_delinq',
            'mths_since_last_major_derog', 'mths_since_last_record',
            'mths_since_rcnt_il', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
            'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
            'pct_tl_nvr_dlq','sec_app_mths_since_last_major_derog']
fill_min = np.setdiff1d(df2.columns.values, np.append(fill_empty, fill_max))


df2[fill_empty] = df2[fill_empty].fillna('')
df2[fill_max] = df2[fill_max].fillna(df2[fill_max].max())
df2[fill_min] = df2[fill_min].fillna(df2[fill_min].min()) 


In [ ]:
#checking again for missing values
nan_mean = df2.isna().mean()
nan_mean = nan_mean[nan_mean != 0].sort_values()
nan_mean

There are some numerical variables which have to be filled with maximum of their respective columns and there are some which have to be filled with min of their column.

For example the 'feature mths_since_last_record' indicates the number of months since the last record (like bankruptcy, foreclosure, tax liens, etc.) so if missing, one should assume that no records were made and the number of months since the "last" record should be a maximum.

 the feature emp_length indicates the number of working years so if missing, one should assume that the borrower never worked and the number of working years should be a minimum.


## Checking for multicollinearity

In [ ]:
#getting numerical values for multicollinearity
numerical = df2.select_dtypes('number').columns.values
df2[numerical].nunique().sort_values()

 We can see that number id has 0 values and id has amount of values equivalent to number of rows so we have to remove these two as they are redundant

In [ ]:
df2 = df2.drop(['num_tl_120dpd_2m', 'id'], axis=1, errors='ignore')

In [ ]:
#calculating collinearity of all the remaining pairs 
num_feat = df2.select_dtypes('number').columns.values
comb_num_feat = np.array(list(combinations(num_feat, 2)))
corr_num_feat = np.array([])
for comb in comb_num_feat:
   corr = pearsonr(df2[comb[0]], df2[comb[1]])[0]
   corr_num_feat = np.append(corr_num_feat, corr)

In [ ]:
#checking for missing values in correlation matrix
corr_num_feat[np.isnan(corr_num_feat)]

In [ ]:
# We will check for values with collinearity above 0.9
high_corr_num = comb_num_feat[np.abs(corr_num_feat) >= 0.9]
high_corr_num


In [ ]:
# We have to remove either of the two values from each list in the array,so removing the first element of every list

df2= df2.drop(np.unique(high_corr_num[:, 0]), axis=1, errors='ignore')

In [ ]:
#Getting categorical variables to calculate correlation

cat_feat = df2.select_dtypes('object').columns.values
df2[cat_feat].nunique().sort_values()

In [ ]:
df2 = df2.drop(['url', 'emp_title'], axis=1, errors='ignore')


Url has as many unique values as the number of rows and emp_title also has a lot of unique values to calculate correlation so removing both of them

Calculating Cramer's V correlation for categorical variables 

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
cat_feat = df2.select_dtypes('object').columns.values
comb_cat_feat = np.array(list(combinations(cat_feat, 2)))
corr_cat_feat = np.array([])
for comb in comb_cat_feat:
    table = pd.pivot_table(df2, values='loan_amnt', index=comb[0], columns=comb[1], aggfunc='count').fillna(0)
    corr = np.sqrt(chi2_contingency(table)[0] / (table.values.sum() * (np.min(table.shape) - 1) ) )
    corr_cat_feat = np.append(corr_cat_feat, corr)
corr_cat_feat    

In [ ]:
high_corr_cat = comb_cat_feat[corr_cat_feat >= 0.9]
high_corr_cat

In [ ]:
#Removing one of the two variables from each list object

df2 = df2.drop(np.unique(high_corr_cat[:, 1]), axis=1, errors='ignore')


## Checking the status of loans in the main data 

In [ ]:
#As we want to classify good loans and bad loans.
# We need to first specify what consists of a good loan and bad loan respectively with the help of loan statuses.

status=pd.DataFrame(df1['loan_status'].value_counts())
status=status.T


In [ ]:
#Assigning good and bad loans based on following assumptions
#1)Current and Fully paid loans are definintely good loans,in Grace period can be considered a good or bad loan
# let's consider it as good for this purpose,rest are all classified as bad

x = df1['loan_status'].copy()
x = x.isin(['Current', 'Fully Paid', 'In Grace Period']).astype('int')
x.value_counts()

## Next step after studying the data and classifying is modelling

The next step is to identify high risk loans based on the parameters in the data since as we had discussed we want to predict loans amoung high risk loans.

There are two features interest rate and the grade which assigns risk to loans.So either one of the features can be used.

In [ ]:
grades = pd.DataFrame(df2['grade'].value_counts()).transpose()
grades.iloc[0]


In [ ]:
plt.bar(status.columns, status.iloc[0], align='center', alpha=0.5)

plt.xlabel('Loan grades')
plt.title('Distribution of loan grades')

In [ ]:
# Getting the grade E loans data as we are predicting good loans out of high risk loans 
# We will be using only these loans for creating the model
df_mod = df2[df2.grade == 'E'].copy()
df_mod.describe()

In [ ]:
df_mod = df_mod.drop([ 'grade','int_rate'], axis=1, errors='ignore')
x_mod = x[df_mod.index]                  #x consists of the column of target values classified on basis of loan status,filtering only grade E loans in the target column

In [ ]:
#splitting into training,validation and testing data
X_train, X_test, y_train, y_test = train_test_split(df_mod, x_mod, stratify=x_mod, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, random_state=0)


In [ ]:
cat_feat_ind = (X_train.dtypes == 'object').to_numpy().nonzero()[0]
pool_train = Pool(X_train, y_train, cat_features=cat_feat_ind)
pool_val = Pool(X_val, y_val, cat_features=cat_feat_ind)
pool_test = Pool(X_test, y_test, cat_features=cat_feat_ind)

In [ ]:
n = y_train.value_counts()
model = CatBoostClassifier(learning_rate=0.03,
                           iterations=1000,
                           early_stopping_rounds=100,
                           class_weights=[1, n[0] / n[1]],
                           verbose=False,
                           random_state=0)
model.fit(pool_train, eval_set=pool_val, plot=True);

Since the validation dataset was used to tune hyperparameters (the number of iterations), I predict targets for the testing dataset which the model hasn't seen yet. The metrics reported here are accuracy, precision and recall. They all have sensible values which is also confirmed by the confusion matrix shown below.

In [ ]:
y_pred_test = model.predict(pool_test)

acc_test = accuracy_score(y_test, y_pred_test)
prec_test = precision_score(y_test, y_pred_test)
rec_test = recall_score(y_test, y_pred_test)
print(f'''Accuracy (test): {acc_test:.3f}
Precision (test): {prec_test:.3f}
Recall (test): {rec_test:.3f}''')

cm = confusion_matrix(y_test, y_pred_test)
ax = sns.heatmap(cm, cmap='viridis_r', annot=True, fmt='d', square=True)
ax.set_xlabel('Predicted')
ax.set_ylabel('True');


# Feature importance


In [ ]:
feat = model.feature_names_
imp = model.feature_importances_
df = pd.DataFrame({'Feature': feat, 'Importance': imp})
df = df.sort_values('Importance', ascending=False)[:10]
sns.barplot(x='Importance', y='Feature', data=df);

In [ ]:
good = df_mod.loc[x_mod == 1, 'loan_amnt']
bad = df_mod.loc[x_mod == 0, 'loan_amnt']

bins = 20
sns.distplot(good, bins=bins, label='Good loans', kde=False, norm_hist=True)
ax = sns.distplot(bad, bins=bins, label='Bad loans', kde=False, norm_hist=True)
ax.set_ylabel('Density')
ax.legend();

In [ ]:
good = df_mod.loc[x_mod == 1, 'mths_since_recent_inq']
bad = df_mod.loc[x_mod == 0, 'mths_since_recent_inq']

bins = 20
sns.distplot(good, bins=bins, label='Good loans', kde=False, norm_hist=True)
ax = sns.distplot(bad, bins=bins, label='Bad loans', kde=False, norm_hist=True)
ax.set_ylabel('Density')
ax.legend();

In [ ]:
good = df_mod.loc[x_mod == 1, 'revol_util']
bad = df_mod.loc[x_mod == 0, 'revol_util']

bins = 20
sns.distplot(good, bins=bins, label='Good loans', kde=False, norm_hist=True)
ax = sns.distplot(bad, bins=bins, label='Bad loans', kde=False, norm_hist=True)
ax.set_ylabel('Density')
ax.legend();

# Model adjustment to increase precision


In [ ]:
y_proba_val = model.predict_proba(pool_val)[:, 1]
p_val, r_val, t_val = precision_recall_curve(y_val, y_proba_val)
plt.plot(r_val, p_val)
plt.xlabel('Recall')
plt.ylabel('Precision');

In [ ]:
p_max = p_val[p_val != 1].max()
t_all = np.insert(t_val, 0, 0)
t_adj_val = t_all[p_val == p_max]
y_adj_val = (y_proba_val > t_adj_val).astype(int)
p_adj_val = precision_score(y_val, y_adj_val)
print(f'Adjusted precision (validation): {p_adj_val:.3f}')

In [ ]:
n = y_adj_val.sum()
ci = proportion_confint(p_adj_val * n, n, alpha=0.05, method='wilson')
print(f'95% confidence interval for adjusted precision: [{ci[0]:.3f}, {ci[1]:.3f}]')

In [ ]:
y_proba_test = model.predict_proba(pool_test)[:, 1]
y_adj_test = (y_proba_test > t_adj_val).astype(int)
p_adj_test = precision_score(y_test, y_adj_test)
r_adj_test = recall_score(y_test, y_adj_test)
print(f'''Adjusted precision (test): {p_adj_test:.3f}
Adjusted recall (test): {r_adj_test:.3f}''')

cm_test = confusion_matrix(y_test, y_adj_test)
ax = sns.heatmap(cm_test, cmap='viridis_r', annot=True, fmt='d', square=True)
ax.set_xlabel('Predicted')
ax.set_ylabel('True');